In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_recall_curve, auc
from sklearn import svm
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [170]:
cons = pd.read_parquet("q2_consDF_final.pqt")
acct = pd.read_parquet("q2_acctDF_final.pqt")
inflows = pd.read_parquet("q2_inflows_final.pqt")
outflows1 = pd.read_parquet ("q2_outflows_1sthalf_final.pqt")
outflows2 = pd.read_parquet ("q2_outflows_2ndhalf_final.pqt")
outflows = pd.concat ([outflows1, outflows2])
cons_holdout = pd. read_parquet("q2_consDF_HOLDOUT_notags_final.pqt")

### Final funcitons

In [185]:
def inflow_over_outlow_features (inflows, outflows):

    # inflow categories and denominator processed
    outflows_consumer_amount = outflows.groupby(["prism_consumer_id"])['amount'].sum().reset_index()
    inflows_cat_amount = inflows.groupby(["prism_consumer_id", "category_description"])['amount'].sum().reset_index()

    # create percentage column
    percent_out_df = pd.merge(outflows_consumer_amount, inflows_cat_amount, on=["prism_consumer_id"], suffixes=('_total_outflows', '_inflow_per_cat'))
    percent_out_df['percentage'] = percent_out_df['amount_inflow_per_cat'] / percent_out_df['amount_total_outflows']

    # using a pivot table to format output
    cat_percent_outflow = percent_out_df.pivot_table(index='prism_consumer_id', columns='category_description', values='percentage', fill_value=0).add_suffix('_inflow_over_outflow')
    cat_percent_outflow.reset_index(inplace=True)
    cat_percent_outflow.drop('UNEMPLOYMENT_BENEFITS_inflow_over_outflow',axis=1,inplace=True)

    return cat_percent_outflow

In [179]:
def inflow_over_inflow_features (inflows):
    
    # inflow categories and denominator processed
    inflows_consumer_amount = inflows.groupby(["prism_consumer_id"])['amount'].sum().reset_index()
    inflows_cat_amount = inflows.groupby(["prism_consumer_id", "category_description"])['amount'].sum().reset_index()

    # create percentage column
    percent_in_df = pd.merge(inflows_consumer_amount, inflows_cat_amount, on=["prism_consumer_id"], suffixes=('_total_inflows', '_inflow_per_cat'))
    percent_in_df['percentage'] = percent_in_df['amount_inflow_per_cat'] / percent_in_df['amount_total_inflows']

    # using a pivot table to format output
    cat_percent_inflow = percent_in_df.pivot_table(index='prism_consumer_id', columns='category_description', values='percentage', fill_value=0).add_suffix('_inflow_over_inflow')
    cat_percent_inflow.reset_index(inplace=True)
    cat_percent_inflow.drop('UNEMPLOYMENT_BENEFITS_inflow_over_inflow',axis=1,inplace=True)

    return cat_percent_inflow

In [180]:
def inflow_over_income_features (inflows, income):

    # inflow categories and denominator processed
    inflows_consumer_amount = inflows.groupby(["prism_consumer_id"])['amount'].sum().reset_index()
    income_final = pd.merge(income,inflows_consumer_amount,on=["prism_consumer_id"], suffixes=('_income', '_inflow'))
    income_final.loc[income_final['amount_income'] == 0, 'amount_income'] = income_final.loc[income_final['amount_income'] == 0, 'amount_inflow']
    income_final.drop('amount_inflow',axis=1,inplace = True)
    inflows_cat_amount = inflows.groupby(["prism_consumer_id", "category_description"])['amount'].sum().reset_index()
    
    # create percentage column
    percent_income_df = pd.merge(income_final, inflows_cat_amount, on=["prism_consumer_id"], suffixes=('_income', '_inflow_per_cat'))
    percent_income_df['percentage'] = percent_income_df['amount'] / percent_income_df['amount_income']
    
    # using a pivot table to format output
    cat_percent_income = percent_income_df.pivot_table(index='prism_consumer_id', columns='category_description', values='percentage', fill_value=0).add_suffix('_inflow_over_income')
    cat_percent_income.reset_index(inplace=True)
    cat_percent_income.drop('UNEMPLOYMENT_BENEFITS_inflow_over_income',axis=1,inplace=True)

    return cat_percent_income

In [181]:
def outflow_over_income_features (inflows, outflows, income):

    # inflow categories and denominator processed
    inflows_consumer_amount = inflows.groupby(["prism_consumer_id"])['amount'].sum().reset_index()
    income_final = pd.merge(income,inflows_consumer_amount,on=["prism_consumer_id"], suffixes=('_income', '_inflow'))
    income_final.loc[income_final['amount_income'] == 0, 'amount_income'] = income_final.loc[income_final['amount_income'] == 0, 'amount_inflow']
    income_final.drop('amount_inflow',axis=1,inplace = True)
    outflows_cat_amount = outflows.groupby(["prism_consumer_id", "category_description"])['amount'].sum().reset_index()

    # create percentage column
    percent_income_df = pd.merge(income_final, outflows_cat_amount, on=["prism_consumer_id"], suffixes=('_income', '_outflow_per_cat'))
    percent_income_df['percentage'] = percent_income_df['amount'] / percent_income_df['amount_income']

    # using a pivot table to format output
    cat_percent_income = percent_income_df.pivot_table(index='prism_consumer_id', columns='category_description', values='percentage', fill_value=0).add_suffix('_outflow_over_income')
    cat_percent_income.reset_index(inplace=True)

    return cat_percent_income

In [186]:
inflow_over_outlow_features(inflows, outflows) 

category_description,prism_consumer_id,DEPOSIT_inflow_over_outflow,EXTERNAL_TRANSFER_inflow_over_outflow,INSURANCE_inflow_over_outflow,INVESTMENT_INCOME_inflow_over_outflow,LOAN_inflow_over_outflow,MISCELLANEOUS_inflow_over_outflow,OTHER_BENEFITS_inflow_over_outflow,PAYCHECK_inflow_over_outflow,PAYCHECK_PLACEHOLDER_inflow_over_outflow,REFUND_inflow_over_outflow,SELF_TRANSFER_inflow_over_outflow,SMALL_DOLLAR_ADVANCE_inflow_over_outflow,TAX_inflow_over_outflow
0,0,0.037499,0.064323,0.0,2.944912e-06,0.000000,0.279697,0.00000,0.003454,0.582320,0.000878,0.162166,0.002727,0.030862
1,2,0.007544,0.159460,0.0,5.629057e-03,0.000000,0.019516,0.00000,0.684885,0.136739,0.000000,0.000000,0.000000,0.000000
2,4,0.090177,0.544602,0.0,3.870215e-07,0.000000,0.096174,0.00000,0.204859,0.000000,0.000000,0.073396,0.000000,0.000000
3,7,0.047455,0.111572,0.0,5.591005e-04,0.142403,0.190807,0.00000,0.472392,0.000000,0.019553,0.025390,0.012772,0.003372
4,9,0.035955,0.002300,0.0,0.000000e+00,0.000000,0.461459,0.00000,0.000000,0.511438,0.000000,0.002884,0.000000,0.029723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,5941,0.004072,0.024894,0.0,2.125515e-02,0.000000,0.074959,0.00000,0.830534,0.000000,0.000790,0.000000,0.000000,0.006152
2972,5943,0.815152,0.242335,0.0,7.212025e-06,0.000000,0.002318,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2973,5944,0.000000,0.304593,0.0,1.995849e-05,0.000000,0.013306,0.00000,0.464381,0.000000,0.185614,0.000000,0.000000,0.000000
2974,5945,0.014354,0.467952,0.0,0.000000e+00,0.037628,0.078893,0.31104,0.000000,0.001291,0.018855,0.002758,0.000000,0.029881


In [183]:
inflow_over_inflow_features(inflows)

category_description,prism_consumer_id,DEPOSIT_inflow_over_inflow,EXTERNAL_TRANSFER_inflow_over_inflow,INSURANCE_inflow_over_inflow,INVESTMENT_INCOME_inflow_over_inflow,LOAN_inflow_over_inflow,MISCELLANEOUS_inflow_over_inflow,OTHER_BENEFITS_inflow_over_inflow,PAYCHECK_inflow_over_inflow,PAYCHECK_PLACEHOLDER_inflow_over_inflow,REFUND_inflow_over_inflow,SELF_TRANSFER_inflow_over_inflow,SMALL_DOLLAR_ADVANCE_inflow_over_inflow,TAX_inflow_over_inflow
0,0,0.032218,0.055264,0.0,2.530144e-06,0.000000,0.240304,0.000000,0.002968,0.500305,0.000755,0.139327,0.002343,0.026515
1,2,0.007441,0.157294,0.0,5.552581e-03,0.000000,0.019251,0.000000,0.675580,0.134881,0.000000,0.000000,0.000000,0.000000
2,4,0.089354,0.539633,0.0,3.834904e-07,0.000000,0.095296,0.000000,0.202989,0.000000,0.000000,0.072727,0.000000,0.000000
3,7,0.046240,0.108716,0.0,5.447859e-04,0.138757,0.185922,0.000000,0.460298,0.000000,0.019052,0.024740,0.012445,0.003286
4,9,0.034448,0.002203,0.0,0.000000e+00,0.000000,0.442113,0.000000,0.000000,0.489996,0.000000,0.002763,0.000000,0.028477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,5941,0.004230,0.025859,0.0,2.207969e-02,0.000000,0.077866,0.000000,0.862753,0.000000,0.000821,0.000000,0.000000,0.006391
2974,5943,0.769148,0.228658,0.0,6.805001e-06,0.000000,0.002187,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2975,5944,0.000000,0.314690,0.0,2.062012e-05,0.000000,0.013747,0.000000,0.479775,0.000000,0.191767,0.000000,0.000000,0.000000
2976,5945,0.014911,0.486107,0.0,0.000000e+00,0.039088,0.081954,0.323108,0.000000,0.001341,0.019586,0.002865,0.000000,0.031040


In [184]:
income = pd.read_csv('income_estimates.csv')
inflow_over_income_features(inflows,income)

category_description,prism_consumer_id,DEPOSIT_inflow_over_income,EXTERNAL_TRANSFER_inflow_over_income,INSURANCE_inflow_over_income,INVESTMENT_INCOME_inflow_over_income,LOAN_inflow_over_income,MISCELLANEOUS_inflow_over_income,OTHER_BENEFITS_inflow_over_income,PAYCHECK_inflow_over_income,PAYCHECK_PLACEHOLDER_inflow_over_income,REFUND_inflow_over_income,SELF_TRANSFER_inflow_over_income,SMALL_DOLLAR_ADVANCE_inflow_over_income,TAX_inflow_over_income
0,0,0.063908,0.109623,0.0,5.018863e-06,0.000000,0.476674,0.000000,0.005887,0.992419,0.001497,0.276372,0.004647,0.052596
1,2,0.009181,0.194070,0.0,6.850784e-03,0.000000,0.023751,0.000000,0.833532,0.166417,0.000000,0.000000,0.000000,0.000000
2,4,0.221256,1.336224,0.0,9.495876e-07,0.000000,0.235970,0.000000,0.502636,0.000000,0.000000,0.180083,0.000000,0.000000
3,7,0.095687,0.224969,0.0,1.127343e-03,0.287135,0.384734,0.000000,0.952509,0.000000,0.039426,0.051194,0.025753,0.006799
4,9,0.037829,0.002420,0.0,0.000000e+00,0.000000,0.485509,0.000000,0.000000,0.538093,0.000000,0.003034,0.000000,0.031272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,5941,0.004899,0.029948,0.0,2.557084e-02,0.000000,0.090178,0.000000,0.999168,0.000000,0.000951,0.000000,0.000000,0.007401
2974,5943,2.353897,0.699785,0.0,2.082600e-05,0.000000,0.006694,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2975,5944,0.000000,0.655912,0.0,4.297871e-05,0.000000,0.028652,0.000000,1.000000,0.000000,0.399702,0.000000,0.000000,0.000000
2976,5945,0.038861,1.266923,0.0,0.000000e+00,0.101874,0.213593,0.842104,0.000000,0.003495,0.051046,0.007467,0.000000,0.080899


In [169]:
outflow_over_income_features(inflows=inflows,outflows=outflows,income=income)

category_description,prism_consumer_id,ACCOUNT_FEES_outflow_over_income,ATM_CASH_outflow_over_income,AUTOMOTIVE_outflow_over_income,BILLS_UTILITIES_outflow_over_income,BNPL_outflow_over_income,CHILD_DEPENDENTS_outflow_over_income,CREDIT_CARD_PAYMENT_outflow_over_income,EDUCATION_outflow_over_income,ENTERTAINMENT_outflow_over_income,...,LOAN_outflow_over_income,MORTGAGE_outflow_over_income,OVERDRAFT_outflow_over_income,PAYCHECK_outflow_over_income,PETS_outflow_over_income,RENT_outflow_over_income,SELF_TRANSFER_outflow_over_income,TAX_outflow_over_income,TRAVEL_outflow_over_income,UNCATEGORIZED_outflow_over_income
0,0,0.000135,0.014081,0.028416,0.0,0.000000,0.0,0.109065,0.000000,0.006388,...,0.339095,0.0,0.000000,0.000000,0.003067,0.000000,0.288454,0.000000,0.007191,0.053970
1,2,0.000000,0.165749,0.041937,0.0,0.000000,0.0,0.087111,0.000000,0.016978,...,0.266176,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117033
2,4,0.000224,0.016727,0.015210,0.0,0.000000,0.0,1.577077,0.000000,0.001696,...,0.125352,0.0,0.000000,0.000000,0.000000,0.000000,0.180083,0.000000,0.000000,0.038434
3,7,0.017788,0.261081,0.012238,0.0,0.003581,0.0,0.341813,0.000000,0.056269,...,0.260445,0.0,0.001795,0.000000,0.002743,0.000000,0.000000,0.027570,0.043876,0.452855
4,9,0.000071,0.020993,0.053290,0.0,0.000000,0.0,0.340449,0.000000,0.005501,...,0.008419,0.0,0.000000,0.000000,0.009310,0.093649,0.005635,0.000000,0.000087,0.116835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,5941,0.000000,0.143834,0.126490,0.0,0.000000,0.0,0.092310,0.000000,0.039207,...,0.020504,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000123,0.000000,0.136110
2972,5943,0.000875,0.216878,0.179535,0.0,0.000000,0.0,1.080600,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002107,0.226170
2973,5944,0.000602,0.005716,0.085599,0.0,0.000000,0.0,0.000000,0.000000,0.014312,...,0.186571,0.0,0.000000,0.035816,0.000000,0.000000,0.000000,0.000000,0.000000,0.068207
2974,5945,0.005624,0.000877,0.080239,0.0,0.000000,0.0,0.000000,0.022822,0.008384,...,0.365438,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.022724,0.009630,0.509051


----------

### Percent outflow per categories over income

In [141]:
inflows_consumer_amount = inflows.groupby(["prism_consumer_id"])['amount'].sum().reset_index()
income_final = pd.merge(income,inflows_consumer_amount,on=["prism_consumer_id"], suffixes=('_income', '_inflow'))
income_final.loc[income_final['amount_income'] == 0, 'amount_income'] = income_final.loc[income_final['amount_income'] == 0, 'amount_inflow']
income_final.drop('amount_inflow',axis=1,inplace = True)
outflows_cat_amount = outflows.groupby(["prism_consumer_id", "category_description"])['amount'].sum().reset_index()


In [145]:
# create percentage column
percent_income_df = pd.merge(income_final, outflows_cat_amount, on=["prism_consumer_id"], suffixes=('_income', '_outflow_per_cat'))
percent_income_df['percentage'] = percent_income_df['amount'] / percent_income_df['amount_income']

# using a pivot table to format output
cat_percent_income = percent_income_df.pivot_table(index='prism_consumer_id', columns='category_description', values='percentage', aggfunc='first', fill_value=0)
cat_percent_income.reset_index(inplace=True)


### Percent inflow per categories over total inflow

In [58]:
inflows_consumer_amount = inflows.groupby(["prism_consumer_id"])['amount'].sum().reset_index()
inflows_cat_amount = inflows.groupby(["prism_consumer_id", "category_description"])['amount'].sum().reset_index()

In [59]:
percent_in_df = pd.merge(inflows_consumer_amount, inflows_cat_amount, on=["prism_consumer_id"], suffixes=('_total_inflows', '_inflow_per_cat'))
percent_in_df['percentage'] = percent_in_df['amount_inflow_per_cat'] / percent_in_df['amount_total_inflows']
percent_in_df

,prism_consumer_id,amount_total_inflows,category_description,amount_inflow_per_cat,percentage
0,0,106713.30,DEPOSIT,3438.07,0.032218
1,0,106713.30,EXTERNAL_TRANSFER,5897.40,0.055264
2,0,106713.30,INVESTMENT_INCOME,0.27,0.000003
3,0,106713.30,MISCELLANEOUS,25643.65,0.240304
4,0,106713.30,PAYCHECK,316.70,0.002968
...,...,...,...,...,...
19247,5949,49864.41,MISCELLANEOUS,39774.30,0.797649
19248,5949,49864.41,REFUND,36.00,0.000722
19249,5949,49864.41,SELF_TRANSFER,2139.56,0.042908
19250,5949,49864.41,SMALL_DOLLAR_ADVANCE,42.56,0.000854


In [126]:
cat_percent_inflow = percent_in_df.pivot_table(index='prism_consumer_id', columns='category_description', values='percentage', aggfunc='first', fill_value=0)
cat_percent_inflow.reset_index(inplace=True)
cat_percent_inflow.drop('UNEMPLOYMENT_BENEFITS',axis=1,inplace=True)

### Percent inflow per categories over total outflow

In [61]:
outflows_consumer_amount = outflows.groupby(["prism_consumer_id"])['amount'].sum().reset_index()
inflows_cat_amount = inflows.groupby(["prism_consumer_id", "category_description"])['amount'].sum().reset_index()

In [64]:
percent_out_df = pd.merge(outflows_consumer_amount, inflows_cat_amount, on=["prism_consumer_id"], suffixes=('_total_outflows', '_inflow_per_cat'))
percent_out_df['percentage'] = percent_out_df['amount_inflow_per_cat'] / percent_out_df['amount_total_outflows']

In [125]:
cat_percent_outflow = percent_out_df.pivot_table(index='prism_consumer_id', columns='category_description', values='percentage', aggfunc='first', fill_value=0)
cat_percent_outflow.reset_index(inplace=True)
cat_percent_outflow.drop('UNEMPLOYMENT_BENEFITS',axis=1,inplace=True)

### Percent inflow per categories over income

In [76]:
income = pd.read_csv('income_estimates.csv')
inflows_cat_amount = inflows.groupby(["prism_consumer_id", "category_description"])['amount'].sum().reset_index()

In [87]:
inflows_consumer_amount = inflows.groupby(["prism_consumer_id"])['amount'].sum().reset_index()
inflows_consumer_amount

,prism_consumer_id,amount
0,0,106713.30
1,2,21512.52
2,4,365067.81
3,7,282441.24
4,9,139339.84
...,...,...
2973,5941,39119.66
2974,5943,42615.72
2975,5944,1454.89
2976,5945,80282.76


In [107]:
income_final = pd.merge(income,inflows_consumer_amount,on=["prism_consumer_id"], suffixes=('_income', '_inflow'))
income_final.loc[income_final['amount_income'] == 0, 'amount_income'] = income_final.loc[income_final['amount_income'] == 0, 'amount_inflow']
income_final.drop('amount_inflow',axis=1,inplace = True)

In [121]:
percent_income_df = pd.merge(income_final, inflows_cat_amount, on=["prism_consumer_id"], suffixes=('_income', '_inflow_per_cat'))
percent_income_df['percentage'] = percent_income_df['amount'] / percent_income_df['amount_income']
percent_income_df

,prism_consumer_id,amount_income,category_description,amount,percentage
0,0,53797.05,DEPOSIT,3438.07,0.063908
1,0,53797.05,EXTERNAL_TRANSFER,5897.40,0.109623
2,0,53797.05,INVESTMENT_INCOME,0.27,0.000005
3,0,53797.05,MISCELLANEOUS,25643.65,0.476674
4,0,53797.05,PAYCHECK,316.70,0.005887
...,...,...,...,...,...
19247,1022,16526.06,DEPOSIT,519.44,0.031432
19248,1022,16526.06,EXTERNAL_TRANSFER,275.00,0.016640
19249,1022,16526.06,INVESTMENT_INCOME,0.04,0.000002
19250,1022,16526.06,MISCELLANEOUS,15728.58,0.951744


In [124]:
cat_percent_income = percent_income_df.pivot_table(index='prism_consumer_id', columns='category_description', values='percentage', aggfunc='first', fill_value=0)
cat_percent_income.reset_index(inplace=True)
cat_percent_income.drop('UNEMPLOYMENT_BENEFITS',axis=1,inplace=True)
cat_percent_income

category_description,prism_consumer_id,DEPOSIT,EXTERNAL_TRANSFER,INSURANCE,INVESTMENT_INCOME,LOAN,MISCELLANEOUS,OTHER_BENEFITS,PAYCHECK,PAYCHECK_PLACEHOLDER,REFUND,SELF_TRANSFER,SMALL_DOLLAR_ADVANCE,TAX
0,0,0.063908,0.109623,0.0,5.018863e-06,0.000000,0.476674,0.000000,0.005887,0.992419,0.001497,0.276372,0.004647,0.052596
1,2,0.009181,0.194070,0.0,6.850784e-03,0.000000,0.023751,0.000000,0.833532,0.166417,0.000000,0.000000,0.000000,0.000000
2,4,0.221256,1.336224,0.0,9.495876e-07,0.000000,0.235970,0.000000,0.502636,0.000000,0.000000,0.180083,0.000000,0.000000
3,7,0.095687,0.224969,0.0,1.127343e-03,0.287135,0.384734,0.000000,0.952509,0.000000,0.039426,0.051194,0.025753,0.006799
4,9,0.037829,0.002420,0.0,0.000000e+00,0.000000,0.485509,0.000000,0.000000,0.538093,0.000000,0.003034,0.000000,0.031272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973,5941,0.004899,0.029948,0.0,2.557084e-02,0.000000,0.090178,0.000000,0.999168,0.000000,0.000951,0.000000,0.000000,0.007401
2974,5943,2.353897,0.699785,0.0,2.082600e-05,0.000000,0.006694,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2975,5944,0.000000,0.655912,0.0,4.297871e-05,0.000000,0.028652,0.000000,1.000000,0.000000,0.399702,0.000000,0.000000,0.000000
2976,5945,0.038861,1.266923,0.0,0.000000e+00,0.101874,0.213593,0.842104,0.000000,0.003495,0.051046,0.007467,0.000000,0.080899
